## ItaData2024
### Intersting rules comparision on various modal trees with different parameters

In [1]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData, SoleBase
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

  Activating project at `~/Documents/Aclai/audio-rules2024`


In [2]:
function mean_longstretch1(x) Catch22.SB_BinaryStats_mean_longstretch1((x)) end
function diff_longstretch0(x) Catch22.SB_BinaryStats_diff_longstretch0((x)) end
function quantile_hh(x) Catch22.SB_MotifThree_quantile_hh((x)) end
function sumdiagcov(x) Catch22.SB_TransitionMatrix_3ac_sumdiagcov((x)) end

function histogramMode_5(x) Catch22.DN_HistogramMode_5((x)) end
function f1ecac(x) Catch22.CO_f1ecac((x)) end
function histogram_even_2_5(x) Catch22.CO_HistogramAMI_even_2_5((x)) end

function get_patched_feature(f::Base.Callable, polarity::Symbol)
    if f in [minimum, maximum, StatsBase.mean, median]
        f
    else
        @eval $(Symbol(string(f)*string(polarity)))
    end
end

features = :catch9
# features = :minmax
# features = :custom

color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36);
r_select = r"\e\[\d+m(.*?)\e\[0m";

nan_guard = [:std, :mean_longstretch1, :diff_longstretch0, :quantile_hh, :sumdiagcov, :histogramMode_5, :f1ecac, :histogram_even_2_5]

for f_name in nan_guard
    @eval (function $(Symbol(string(f_name)*"+"))(channel)
        val = $(f_name)(channel)

        if isnan(val)
            SoleData.aggregator_bottom(SoleData.existential_aggregator(≥), eltype(channel))
        else
            eltype(channel)(val)
        end
    end)
    @eval (function $(Symbol(string(f_name)*"-"))(channel)
        val = $(f_name)(channel)

        if isnan(val)
            SoleData.aggregator_bottom(SoleData.existential_aggregator(≤), eltype(channel))
        else
            eltype(channel)(val)
        end
    end)
end

if features == :catch9
    metaconditions = [
        (≥, get_patched_feature(maximum, :+)),            (≤, get_patched_feature(maximum, :-)),
        (≥, get_patched_feature(minimum, :+)),            (≤, get_patched_feature(minimum, :-)),
        (≥, get_patched_feature(StatsBase.mean, :+)),     (≤, get_patched_feature(StatsBase.mean, :-)),
        (≥, get_patched_feature(median, :+)),             (≤, get_patched_feature(median, :-)),
        (≥, get_patched_feature(std, :+)),                (≤, get_patched_feature(std, :-)),
        (≥, get_patched_feature(mean_longstretch1, :+)),  (≤, get_patched_feature(mean_longstretch1, :-)),
        (≥, get_patched_feature(diff_longstretch0, :+)),  (≤, get_patched_feature(diff_longstretch0, :-)),
        (≥, get_patched_feature(quantile_hh, :+)),        (≤, get_patched_feature(quantile_hh, :-)),
        (≥, get_patched_feature(sumdiagcov, :+)),         (≤, get_patched_feature(sumdiagcov, :-)),
    ]
elseif features == :minmax
    metaconditions = [
        (≥, get_patched_feature(maximum, :+)),            (≤, get_patched_feature(maximum, :-)),
        (≥, get_patched_feature(minimum, :+)),            (≤, get_patched_feature(minimum, :-)),
    ]
elseif features == :custom
    metaconditions = [
        (≥, get_patched_feature(maximum, :+)),            (≤, get_patched_feature(maximum, :-)),
        # (≥, get_patched_feature(minimum, :+)),            (≤, get_patched_feature(minimum, :-)),
        # (≥, get_patched_feature(StatsBase.mean, :+)),     (≤, get_patched_feature(StatsBase.mean, :-)),
        # (≥, get_patched_feature(median, :+)),             (≤, get_patched_feature(median, :-)),
        (≥, get_patched_feature(std, :+)),                (≤, get_patched_feature(std, :-)),
        # (≥, get_patched_feature(mean_longstretch1, :+)),  (≤, get_patched_feature(mean_longstretch1, :-)),
        # (≥, get_patched_feature(diff_longstretch0, :+)),  (≤, get_patched_feature(diff_longstretch0, :-)),
        # (≥, get_patched_feature(quantile_hh, :+)),        (≤, get_patched_feature(quantile_hh, :-)),
        # (≥, get_patched_feature(sumdiagcov, :+)),         (≤, get_patched_feature(sumdiagcov, :-)),
        (≥, get_patched_feature(histogramMode_5, :+)),    (≤, get_patched_feature(histogramMode_5, :-)),
        (≥, get_patched_feature(f1ecac, :+)),             (≤, get_patched_feature(f1ecac, :-)),
        (≥, get_patched_feature(histogram_even_2_5, :+)), (≤, get_patched_feature(histogram_even_2_5, :-)),
    ]
else
    error("Unknown set of features: $features.")
end
;

### Pneumonia
scale: semitones without mfcc

In [3]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:56


▣ ⟨G⟩((max[cntrd] ≥ 1084.33) ∧ (std⁻[skwns] ≤ 0.08))  ↣  Healthy : (ninstances = 101, ncovered = 17, coverage = 0.17, confidence = 1.0, lift = 2.02, natoms = 2, height = 2)
▣ ⟨G⟩(max[mel8=1194Hz] ≤ -4.91) ∧ [G](max[cntrd] < 1084.33) ∧ [G](std⁺[mel10=1687Hz] < 0.42) ∧ [G]((max[mel8=1194Hz] ≤ -4.91) → [DBE](std⁺[mel13=2832Hz] < 0.17))  ↣  Pneumonia : (ninstances = 101, ncovered = 28, coverage = 0.28, confidence = 0.93, lift = 1.84, natoms = 5, height = 4)


### Pneumonia
scale: semitones with mfcc

In [4]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:56


▣ ⟨G⟩((max[mfcc3] ≤ -0.52) ∧ ⟨L⟩(max[crest] ≤ 20.89)) ∧ [G](max[mfcc2] > 1.6) ∧ [G]((max[mfcc3] ≤ -0.52) → [D̅B̅E̅](max[mfcc4] > -0.53)) ∧ [G]((max[mfcc3] ≤ -0.52) → [DBE](max[flatn] < 0.1)) ∧ [G]((max[mfcc3] ≤ -0.52) → [G](sumdiagcov⁻[mel11=2005Hz] > 0.0)) ∧ [G]((max[mfcc3] ≤ -0.52) → [L]((max[crest] ≤ 20.89) → (std⁻[mel6=845Hz] > 0.0)))  ↣  Healthy : (ninstances = 101, ncovered = 12, coverage = 0.12, confidence = 0.58, lift = 1.05, natoms = 12, height = 8)
▣ [G](max[mfcc2] > 1.6) ∧ [G](max[mfcc3] > -0.52) ∧ [G](max[mel12=2383Hz] > 0.0) ∧ [G](std⁺[mfcc6] < 0.32)  ↣  Pneumonia : (ninstances = 101, ncovered = 25, coverage = 0.25, confidence = 0.96, lift = 2.15, natoms = 4, height = 4)
▣ ⟨G⟩((max[mfcc2] ≤ 1.6) ∧ ⟨D̅B̅E̅⟩(min[mfcc5] ≥ -0.01)) ∧ [G]((max[mfcc2] ≤ 1.6) → [G](max[mfcc5] < 0.91))  ↣  Healthy : (ninstances = 101, ncovered = 40, coverage = 0.4, confidence = 0.92, lift = 1.67, natoms = 4, height = 4)


### Pneumonia
scale: mel_htk without mfcc

In [5]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X11sZmlsZQ==.jl:56


▣ ⟨G⟩((max[cntrd] ≥ 1099.63) ∧ ⟨AO⟩(max[decrs] ≥ -0.04))  ↣  Healthy : (ninstances = 101, ncovered = 24, coverage = 0.24, confidence = 0.96, lift = 1.98, natoms = 2, height = 3)
▣ ⟨G⟩((max[mel11=1179Hz] ≤ -4.59) ∧ ⟨L⟩(std⁻[mel22=2829Hz] ≤ 0.02)) ∧ [G](max[cntrd] < 1099.63) ∧ [G](std⁺[mel15=1663Hz] < 0.47) ∧ [G]((max[mel11=1179Hz] ≤ -4.59) → [AO](min[skwns] > 1.42)) ∧ [G]((max[mel11=1179Hz] ≤ -4.59) → [L]((std⁻[mel22=2829Hz] ≤ 0.02) → (std⁻[mel6=710Hz] > 0.0)))  ↣  Pneumonia : (ninstances = 101, ncovered = 31, coverage = 0.31, confidence = 0.94, lift = 1.82, natoms = 9, height = 7)


### Pneumonia
scale: mel_htk with mfcc

In [6]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X13sZmlsZQ==.jl:56


▣ ⟨G⟩((max[mfcc3] ≤ 0.07) ∧ ⟨A̅O̅⟩(min[mfcc2] ≤ 2.06))  ↣  Healthy : (ninstances = 101, ncovered = 25, coverage = 0.25, confidence = 1.0, lift = 2.06, natoms = 2, height = 3)
▣ [G](max[mfcc3] > 0.07) ∧ [G](max[mfcc2] > 1.93) ∧ [G](max[mel10=1074Hz] > 0.0) ∧ [G](mean_longstretch1⁺[mel4=558Hz] < 9.0)  ↣  Pneumonia : (ninstances = 101, ncovered = 37, coverage = 0.37, confidence = 0.97, lift = 1.89, natoms = 4, height = 4)


### Bronchiectasis
scale: semitones without mfcc

In [7]:
# experiment = :Pneumonia
experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:56


▣ [G](max[mel7=1005Hz] < -4.14) ∧ [G](std⁻[mel13=2832Hz] > 0.0)  ↣  Healthy : (ninstances = 36, ncovered = 6, coverage = 0.17, confidence = 0.67, lift = 1.33, natoms = 2, height = 2)
▣ ⟨G⟩((max[mel7=1005Hz] ≥ -4.14) ∧ ⟨AO⟩((max[kurts] ≤ 6.44) ∧ ⟨D̅B̅E̅⟩((mean[mel7=1005Hz] ≤ -4.18) ∧ ⟨D̅B̅E̅⟩((median[decrs] ≥ -0.36) ∧ (std⁺[mel1=357Hz] ≥ 0.48)))))  ↣  Healthy : (ninstances = 36, ncovered = 6, coverage = 0.17, confidence = 1.0, lift = 2.0, natoms = 5, height = 8)
▣ ⟨G⟩((max[mel7=1005Hz] ≥ -4.14) ∧ ⟨AO⟩(max[kurts] ≤ 6.44)) ∧ [G]((max[mel7=1005Hz] ≥ -4.14) → [AO]((max[kurts] ≤ 6.44) → [D̅B̅E̅](mean[mel7=1005Hz] > -4.18))) ∧ [G]((max[mel7=1005Hz] ≥ -4.14) → [AO]((max[kurts] ≤ 6.44) → (max[mel11=2005Hz] > -5.05)))  ↣  Bronchiectasis : (ninstances = 36, ncovered = 6, coverage = 0.17, confidence = 1.0, lift = 2.0, natoms = 8, height = 7)
▣ ⟨G⟩(max[mel7=1005Hz] ≥ -4.14) ∧ [G]((max[mel7=1005Hz] ≥ -4.14) → [AO](max[kurts] > 6.44))  ↣  Bronchiectasis : (ninstances = 36, ncovered = 10, coverage = 0

### Bronchiectasis
scale: semitones with mfcc

In [8]:
# experiment = :Pneumonia
experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:56


▣ ⟨G⟩(max[kurts] ≤ 5.06) ∧ [G]((max[kurts] ≤ 5.06) → [D̅B̅E̅](median[mel8=1194Hz] > 0.0))  ↣  Bronchiectasis : (ninstances = 36, ncovered = 5, coverage = 0.14, confidence = 1.0, lift = 2.25, natoms = 3, height = 4)
▣ ⟨G⟩(std⁺[mel2=424Hz] ≥ 0.01) ∧ [G](max[kurts] > 5.06) ∧ [G](max[mel7=1005Hz] > 0.0)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 6, coverage = 0.17, confidence = 0.83, lift = 1.88, natoms = 3, height = 3)
▣ ⟨G⟩((max[kurts] ≤ 5.06) ∧ ⟨D̅B̅E̅⟩(median[mel8=1194Hz] ≤ 0.0)) ∧ [G]((max[kurts] ≤ 5.06) → [D̅B̅E̅]((median[mel8=1194Hz] ≤ 0.0) → (quantile_hh⁺[mel9=1419Hz] < 2.2)))  ↣  Healthy : (ninstances = 36, ncovered = 16, coverage = 0.44, confidence = 0.94, lift = 1.69, natoms = 5, height = 5)


### Bronchiectasis
scale: mel_htk without mfcc

In [9]:
# experiment = :Pneumonia
experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X22sZmlsZQ==.jl:56


▣ ⟨G⟩(max[flux] ≤ 0.01) ∧ [G]((max[flux] ≤ 0.01) → [G](max[kurts] > 5.19)) ∧ [G]((max[flux] ≤ 0.01) → [D̅B̅E̅](max[mel10=1074Hz] > -5.53)) ∧ [G]((max[flux] ≤ 0.01) → (max[mel9=975Hz] < -4.67))  ↣  Bronchiectasis : (ninstances = 36, ncovered = 5, coverage = 0.14, confidence = 1.0, lift = 1.8, natoms = 7, height = 6)
▣ ⟨G⟩((max[flux] ≤ 0.01) ∧ ⟨G⟩(max[kurts] ≤ 5.19)) ∧ [G]((max[flux] ≤ 0.01) → [G]((max[kurts] ≤ 5.19) → (sumdiagcov⁻[mel1=359Hz] > 0.01)))  ↣  Healthy : (ninstances = 36, ncovered = 8, coverage = 0.22, confidence = 1.0, lift = 2.25, natoms = 5, height = 5)
▣ ⟨G⟩(std⁺[flatn] ≥ 0.01) ∧ [G](max[flux] > 0.01) ∧ [G](std⁺[sprd] < 197.9)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 13, coverage = 0.36, confidence = 0.92, lift = 1.66, natoms = 3, height = 3)


### Bronchiectasis
scale: mel_htk with mfcc

In [10]:
# experiment = :Pneumonia
experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:56


▣ ⟨G⟩(max[mfcc4] ≥ 0.43) ∧ [G](std⁺[mel2=421Hz] < 0.02) ∧ [G]((max[mfcc4] ≥ 0.43) → [D̅B̅E̅](max[mfcc6] > 0.6)) ∧ [G]((max[mfcc4] ≥ 0.43) → (min[mel1=359Hz] < 0.02))  ↣  Bronchiectasis : (ninstances = 36, ncovered = 4, coverage = 0.11, confidence = 1.0, lift = 2.12, natoms = 6, height = 5)
▣ ⟨G⟩(mean_longstretch1⁺[mel1=359Hz] ≥ 4.0) ∧ [G](std⁺[mel2=421Hz] < 0.02) ∧ [G](max[mfcc4] < 0.43) ∧ [G](max[mfcc7] > -0.79)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 5, coverage = 0.14, confidence = 0.8, lift = 1.69, natoms = 4, height = 4)
▣ ⟨G⟩((std⁺[mel2=421Hz] ≥ 0.02) ∧ (min[mel1=359Hz] ≥ 0.0))  ↣  Bronchiectasis : (ninstances = 36, ncovered = 9, coverage = 0.25, confidence = 1.0, lift = 2.12, natoms = 2, height = 2)
▣ ⟨G⟩((max[mfcc4] ≥ 0.43) ∧ ⟨D̅B̅E̅⟩((max[mfcc6] ≤ 0.6) ∧ ⟨AO⟩(min[mfcc5] ≥ 0.34))) ∧ [G](std⁺[mel2=421Hz] < 0.02)  ↣  Healthy : (ninstances = 36, ncovered = 16, coverage = 0.44, confidence = 1.0, lift = 1.89, natoms = 4, height = 6)


### COPD
scale: semitones without mfcc

In [11]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X26sZmlsZQ==.jl:56


▣ ⟨G⟩((max[mel1=357Hz] ≤ -3.83) ∧ ⟨L̅⟩((max[mel13=2832Hz] ≥ -8.07) ∧ (max[skwns] ≤ 1.57) ∧ ⟨D̅B̅E̅⟩(std⁻[cntrd] ≤ 13.24))) ∧ [G]((max[mel1=357Hz] ≤ -3.83) → [L̅]((max[mel13=2832Hz] ≥ -8.07) → (max[skwns] ≤ 1.57) → [G](std⁺[entrp] < 0.11))) ∧ [G]((max[mel1=357Hz] ≤ -3.83) → [L̅]((max[mel13=2832Hz] ≥ -8.07) → (max[skwns] ≤ 1.57) → [D̅B̅E̅]((std⁻[cntrd] ≤ 13.24) → (std⁺[mel2=424Hz] < 0.41))))  ↣  Healthy : (ninstances = 110, ncovered = 11, coverage = 0.1, confidence = 0.91, lift = 1.85, natoms = 13, height = 8)
▣ ⟨G⟩(max[mel1=357Hz] ≤ -3.83) ∧ [G]((max[mel1=357Hz] ≤ -3.83) → [L̅](max[mel13=2832Hz] < -8.07))  ↣  COPD : (ninstances = 110, ncovered = 13, coverage = 0.12, confidence = 1.0, lift = 1.96, natoms = 3, height = 4)
▣ [G](max[mel1=357Hz] > -3.83) ∧ [G](max[mel12=2383Hz] < -4.46) ∧ [G](max[cntrd] < 986.78) ∧ [G](max[mel7=1005Hz] < -2.36)  ↣  COPD : (ninstances = 110, ncovered = 14, coverage = 0.13, confidence = 0.93, lift = 1.82, natoms = 4, height = 4)
▣ ⟨G⟩(max[mel12=2383Hz] ≥ -4.4

### COPD
scale: semitones with mfcc

In [12]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X31sZmlsZQ==.jl:56


▣ ⟨G⟩((max[mfcc6] ≥ 0.05) ∧ (max[mfcc5] ≤ 0.24)) ∧ [G](max[mel5=711Hz] > 0.0)  ↣  COPD : (ninstances = 110, ncovered = 28, coverage = 0.25, confidence = 0.96, lift = 2.04, natoms = 3, height = 3)
▣ ⟨G⟩((max[mel5=711Hz] ≤ 0.0) ∧ ⟨AO⟩((min[cntrd] ≥ 984.58) ∧ ⟨AO⟩(min[mfcc4] ≥ 0.44))) ∧ [G]((max[mel5=711Hz] ≤ 0.0) → [AO]((min[cntrd] ≥ 984.58) → [AO]((min[mfcc4] ≥ 0.44) → [G](max[mel10=1687Hz] > 0.0))))  ↣  Healthy : (ninstances = 110, ncovered = 34, coverage = 0.31, confidence = 0.97, lift = 1.84, natoms = 7, height = 8)


### COPD
scale: mel_htk without mfcc

In [13]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X33sZmlsZQ==.jl:56


▣ ⟨G⟩(max[mel4=558Hz] ≤ -4.45) ∧ [G]((max[mel4=558Hz] ≤ -4.45) → (max[mel21=2632Hz] < -7.98))  ↣  COPD : (ninstances = 110, ncovered = 19, coverage = 0.17, confidence = 0.95, lift = 1.56, natoms = 3, height = 3)
▣ ⟨G⟩((max[mel4=558Hz] ≤ -4.45) ∧ (max[mel21=2632Hz] ≥ -7.98) ∧ ⟨A̅O̅⟩(max[skwns] ≤ 1.44)) ∧ [G]((max[mel4=558Hz] ≤ -4.45) → (max[mel21=2632Hz] ≥ -7.98) → [A̅O̅]((max[skwns] ≤ 1.44) → [DBE](sumdiagcov⁻[mel24=3257Hz] > 0.01)))  ↣  Healthy : (ninstances = 110, ncovered = 24, coverage = 0.22, confidence = 0.96, lift = 2.45, natoms = 7, height = 7)
▣ [G](max[mel4=558Hz] > -4.45) ∧ [G](max[mel18=2106Hz] > -6.18) ∧ [G](max[mel2=421Hz] > -3.86)  ↣  COPD : (ninstances = 110, ncovered = 31, coverage = 0.28, confidence = 0.97, lift = 1.59, natoms = 3, height = 3)


### COPD
scale: mel_htk with mfcc

In [14]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X35sZmlsZQ==.jl:56


▣ ⟨G⟩((max[mel4=558Hz] ≤ 0.0) ∧ ⟨D̅B̅E̅⟩((quantile_hh⁺[mel20=2447Hz] ≥ 1.89) ∧ (std⁻[flatn] ≤ 0.05))) ∧ [G]((max[mel4=558Hz] ≤ 0.0) → [D̅B̅E̅]((quantile_hh⁺[mel20=2447Hz] ≥ 1.89) → [AO](mean[mfcc3] > 0.12))) ∧ [G]((max[mel4=558Hz] ≤ 0.0) → [D̅B̅E̅]((quantile_hh⁺[mel20=2447Hz] ≥ 1.89) → [A̅O̅](min[mel3=488Hz] > 0.0)))  ↣  COPD : (ninstances = 110, ncovered = 11, coverage = 0.1, confidence = 0.64, lift = 1.3, natoms = 9, height = 7)
▣ ⟨G⟩(max[mfcc7] ≥ -0.26) ∧ [G](max[mel4=558Hz] > 0.0) ∧ [G]((max[mfcc7] ≥ -0.26) → [D̅B̅E̅](mean[mel2=421Hz] > 0.0))  ↣  COPD : (ninstances = 110, ncovered = 29, coverage = 0.26, confidence = 0.97, lift = 1.97, natoms = 4, height = 4)
▣ ⟨G⟩((max[mel4=558Hz] ≤ 0.0) ∧ ⟨D̅B̅E̅⟩((quantile_hh⁺[mel20=2447Hz] ≥ 1.89) ∧ ⟨AO⟩((mean[mfcc3] ≤ 0.12) ∧ ⟨AO⟩(std⁻[mel1=359Hz] ≤ 0.0)))) ∧ [G]((max[mel4=558Hz] ≤ 0.0) → [D̅B̅E̅]((quantile_hh⁺[mel20=2447Hz] ≥ 1.89) → [AO]((mean[mfcc3] ≤ 0.12) → [AO]((std⁻[mel1=359Hz] ≤ 0.0) → (median[mfcc13] > -0.37)))))  ↣  Healthy : (ninstan

### URTI
scale: semitones without mfcc

In [15]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X40sZmlsZQ==.jl:56


▣ ⟨G⟩(max[flatn] ≥ 0.14) ∧ [G]((max[flatn] ≥ 0.14) → [L](max[flatn] < 0.28)) ∧ [G]((max[flatn] ≥ 0.14) → [L̅](sumdiagcov⁻[flatn] > 0.01)) ∧ [G]((max[flatn] ≥ 0.14) → [D̅B̅E̅](max[crest] > 15.22))  ↣  Healthy : (ninstances = 86, ncovered = 12, coverage = 0.14, confidence = 0.5, lift = 1.08, natoms = 7, height = 6)
▣ ⟨G⟩((max[mel7=1005Hz] ≤ -4.79) ∧ ⟨AO⟩(sumdiagcov⁻[entrp] ≤ 0.04)) ∧ [G](max[flatn] < 0.14) ∧ [G]((max[mel7=1005Hz] ≤ -4.79) → [D̅B̅E̅](max[mel8=1194Hz] > -6.82)) ∧ [G]((max[mel7=1005Hz] ≤ -4.79) → [A̅O̅](mean_longstretch1⁺[mel4=599Hz] < 8.0))  ↣  URTI : (ninstances = 86, ncovered = 21, coverage = 0.24, confidence = 0.9, lift = 1.69, natoms = 7, height = 6)


### URTI
scale: semitones with mfcc

In [16]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
experiment = :URTI
# experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X42sZmlsZQ==.jl:56


▣ ⟨G⟩((max[mfcc2] ≥ 5.02) ∧ ⟨A̅O̅⟩(std⁻[sprd] ≤ 1.03))  ↣  URTI : (ninstances = 86, ncovered = 9, coverage = 0.1, confidence = 0.78, lift = 1.37, natoms = 2, height = 3)
▣ ⟨G⟩(max[skwns] ≤ 1.13) ∧ [G](max[mfcc2] < 5.02) ∧ [G](max[mfcc4] > -0.03) ∧ [G](std⁺[skwns] < 3.45) ∧ [G](std⁺[mfcc4] < 0.61) ∧ [G]((max[skwns] ≤ 1.13) → [A̅O̅](max[cntrd] > 690.0)) ∧ [G]((max[skwns] ≤ 1.13) → [G](sumdiagcov⁻[mel3=504Hz] > 0.0))  ↣  Healthy : (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.5, lift = 1.16, natoms = 9, height = 7)
▣ ⟨G⟩((max[mfcc2] ≥ 5.02) ∧ (mean[flatn] ≥ 0.02) ∧ ⟨G⟩(std⁺[entrp] ≥ 0.1)) ∧ [G]((max[mfcc2] ≥ 5.02) → [A̅O̅](std⁻[sprd] > 1.03)) ∧ [G]((max[mfcc2] ≥ 5.02) → [L](std⁺[mel13=2832Hz] < 0.0)) ∧ [G]((max[mfcc2] ≥ 5.02) → (mean[flatn] ≥ 0.02) → [L](std⁻[mel2=424Hz] > 0.0))  ↣  URTI : (ninstances = 86, ncovered = 12, coverage = 0.14, confidence = 0.83, lift = 1.46, natoms = 10, height = 7)


### URTI
scale: mel_htk without mfcc

In [17]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X44sZmlsZQ==.jl:56


▣ ⟨G⟩((max[rllff] ≥ 2156.25) ∧ (sumdiagcov⁻[mel1=359Hz] ≤ 0.04)) ∧ [G]((max[rllff] ≥ 2156.25) → [L](std⁺[slope] < 0.0))  ↣  Healthy : (ninstances = 86, ncovered = 11, coverage = 0.13, confidence = 0.73, lift = 1.42, natoms = 4, height = 4)
▣ ⟨G⟩(max[mel11=1179Hz] ≤ -4.56) ∧ [G](max[rllff] < 2156.25) ∧ [G]((max[mel11=1179Hz] ≤ -4.56) → [A̅O̅](max[mel9=975Hz] > -6.37)) ∧ [G]((max[mel11=1179Hz] ≤ -4.56) → [G](max[mel15=1663Hz] < -4.4))  ↣  URTI : (ninstances = 86, ncovered = 14, coverage = 0.16, confidence = 0.86, lift = 1.76, natoms = 6, height = 5)


### URTI
scale: mel_htk with mfcc

In [18]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
experiment = :URTI
# experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X46sZmlsZQ==.jl:56


▣ ⟨G⟩((max[mfcc2] ≤ 2.89) ∧ ⟨D̅B̅E̅⟩((mean[mfcc13] ≤ 0.0) ∧ ⟨G⟩(std⁺[mel8=882Hz] ≥ 0.0))) ∧ [G]((max[mfcc2] ≤ 2.89) → [DBE](max[mfcc4] < 0.57)) ∧ [G]((max[mfcc2] ≤ 2.89) → [D̅B̅E̅]((mean[mfcc13] ≤ 0.0) → (mean[mfcc4] > -0.44))) ∧ [G]((max[mfcc2] ≤ 2.89) → [D̅B̅E̅]((mean[mfcc13] ≤ 0.0) → [G]((std⁺[mel8=882Hz] ≥ 0.0) → [A̅O̅](mean[mfcc11] < 0.3)))) ∧ [G]((max[mfcc2] ≤ 2.89) → [D̅B̅E̅]((mean[mfcc13] ≤ 0.0) → [G]((std⁺[mel8=882Hz] ≥ 0.0) → (quantile_hh⁺[mel2=421Hz] < 2.17))))  ↣  URTI : (ninstances = 86, ncovered = 9, coverage = 0.1, confidence = 0.67, lift = 1.33, natoms = 16, height = 9)
▣ ⟨G⟩((max[mel11=1179Hz] ≤ 0.0) ∧ ⟨AO⟩((min[mfcc10] ≥ 0.28) ∧ (max[mel1=359Hz] ≤ 0.0))) ∧ [G](max[mfcc2] > 2.89) ∧ [G]((max[mel11=1179Hz] ≤ 0.0) → (max[mel13=1407Hz] > 0.0))  ↣  URTI : (ninstances = 86, ncovered = 14, coverage = 0.16, confidence = 0.86, lift = 1.71, natoms = 6, height = 6)
▣ ⟨G⟩((max[mfcc2] ≤ 2.89) ∧ ⟨DBE⟩(max[mfcc4] ≥ 0.57)) ∧ [G]((max[mfcc2] ≤ 2.89) → [DBE]((max[mfcc4] ≥ 0.57) → [L](st

### Bronchiolitis
scale: semitones without mfcc

In [19]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X51sZmlsZQ==.jl:56


▣ [G](max[crest] < 39.35) ∧ [G](sumdiagcov⁻[entrp] > 0.0)  ↣  Healthy : (ninstances = 56, ncovered = 9, coverage = 0.16, confidence = 1.0, lift = 1.93, natoms = 2, height = 2)
▣ ⟨G⟩((max[crest] ≥ 39.35) ∧ ⟨AO⟩((median[flux] ≥ 0.21) ∧ ⟨AO⟩((min[mel4=599Hz] ≤ -5.09) ∧ (max[mel8=1194Hz] ≥ -5.55)))) ∧ [G]((max[crest] ≥ 39.35) → [AO]((median[flux] ≥ 0.21) → [AO]((min[mel4=599Hz] ≤ -5.09) → [L](max[crest] > 15.83))))  ↣  Bronchiolitis : (ninstances = 56, ncovered = 17, coverage = 0.3, confidence = 0.88, lift = 1.83, natoms = 8, height = 8)


### Bronchiolitis
scale: semitones with mfcc

In [20]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
experiment = :Bronchiolitis

scale = :semitones
# scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X53sZmlsZQ==.jl:56


▣ [G](max[crest] < 39.35) ∧ [G](std⁻[mel11=2005Hz] > 0.0)  ↣  Healthy : (ninstances = 56, ncovered = 12, coverage = 0.21, confidence = 0.83, lift = 1.94, natoms = 2, height = 2)
▣ ⟨G⟩((max[crest] ≥ 39.35) ∧ ⟨AO⟩((mean[mel1=357Hz] ≥ 0.0) ∧ (mean[mfcc1] ≤ -19.99))) ∧ [G]((max[crest] ≥ 39.35) → [AO]((mean[mel1=357Hz] ≥ 0.0) → (mean[mfcc1] ≤ -19.99) → [AO](min[mel3=504Hz] > 0.0)))  ↣  Bronchiolitis : (ninstances = 56, ncovered = 14, coverage = 0.25, confidence = 0.93, lift = 1.63, natoms = 7, height = 7)


### Bronchiolitis
scale: mel_htk without mfcc

In [21]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

featset = ()
# featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X55sZmlsZQ==.jl:56


▣ ⟨G⟩((max[cntrd] ≤ 489.66) ∧ ⟨A̅O̅⟩((min[skwns] ≤ 2.99) ∧ ⟨L⟩(std⁺[mel2=421Hz] ≥ 0.7))) ∧ [G]((max[cntrd] ≤ 489.66) → [A̅O̅]((min[skwns] ≤ 2.99) → [L̅](std⁺[mel13=1407Hz] < 0.52))) ∧ [G]((max[cntrd] ≤ 489.66) → [A̅O̅]((min[skwns] ≤ 2.99) → [L](max[mel3=488Hz] > -6.22))) ∧ [G]((max[cntrd] ≤ 489.66) → [A̅O̅]((min[skwns] ≤ 2.99) → [L]((std⁺[mel2=421Hz] ≥ 0.7) → (std⁻[mel4=558Hz] > 0.16)))) ∧ [G]((max[cntrd] ≤ 489.66) → [A̅O̅]((min[skwns] ≤ 2.99) → [L]((std⁺[mel2=421Hz] ≥ 0.7) → (min[mel1=359Hz] > -5.48))))  ↣  Bronchiolitis : (ninstances = 56, ncovered = 7, coverage = 0.12, confidence = 0.86, lift = 1.85, natoms = 17, height = 9)
▣ ⟨G⟩((max[cntrd] ≤ 489.66) ∧ ⟨G⟩(std⁻[mel3=488Hz] ≤ 0.02)) ∧ [G]((max[cntrd] ≤ 489.66) → [A̅O̅](min[skwns] > 2.99))  ↣  Healthy : (ninstances = 56, ncovered = 7, coverage = 0.12, confidence = 1.0, lift = 1.87, natoms = 4, height = 4)
▣ ⟨G⟩(std⁻[mel3=488Hz] ≤ 0.01) ∧ [G](max[cntrd] > 489.66)  ↣  Healthy : (ninstances = 56, ncovered = 9, coverage = 0.16, confiden

### Bronchiolitis
scale: mel_htk with mfcc

In [22]:
# experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
experiment = :Bronchiolitis

# scale = :semitones
scale = :mel_htk

# featset = ()
featset = (:mfcc,)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment in avail_exp "Unknown type of experiment: $experiment."

destpath = "results/modal/$scale"
:mfcc in featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
    variable_names_map=variable_names
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)

readmetrics.(interesting_rules)

printmodel.(
    sort(interesting_rules, by = readmetrics); 
    show_metrics = (; round_digits = 2, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), 
    syntaxstring_kwargs = (; threshold_digits = 2), 
    variable_names_map=variable_names
);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X60sZmlsZQ==.jl:56


▣ ⟨G⟩(quantile_hh⁺[decrs] ≥ 2.16) ∧ [G](max[cntrd] > 489.66) ∧ [G](max[mfcc8] > -0.17)  ↣  Healthy : (ninstances = 56, ncovered = 7, coverage = 0.12, confidence = 0.71, lift = 1.6, natoms = 3, height = 3)
▣ ⟨G⟩(max[cntrd] ≤ 489.66) ∧ [G]((max[cntrd] ≤ 489.66) → [L̅](max[mfcc3] > 0.09)) ∧ [G]((max[cntrd] ≤ 489.66) → [A̅O̅](median[crest] > 29.84))  ↣  Healthy : (ninstances = 56, ncovered = 8, coverage = 0.14, confidence = 0.5, lift = 1.12, natoms = 5, height = 5)
▣ ⟨G⟩(max[mfcc8] ≤ -0.17) ∧ [G](max[cntrd] > 489.66)  ↣  Healthy : (ninstances = 56, ncovered = 8, coverage = 0.14, confidence = 0.88, lift = 1.96, natoms = 2, height = 2)
▣ ⟨G⟩((max[cntrd] ≤ 489.66) ∧ ⟨L̅⟩((max[mfcc3] ≤ 0.09) ∧ ⟨G⟩((max[mel1=359Hz] ≥ 0.0) ∧ ⟨AO⟩((max[mel11=1179Hz] ≤ 0.0) ∧ (min[decrs] ≤ -1.46)))))  ↣  Bronchiolitis : (ninstances = 56, ncovered = 20, coverage = 0.36, confidence = 0.8, lift = 1.45, natoms = 5, height = 8)
